In [1]:
# CS 124: Machine Learning in Genetics
# Project: Haplotype Phaser
# Contributors: Aditya Pimplaskar, Aditya Joglekar
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [2]:
# Imputation functions
def fillna(col):
    if col.value_counts().index[0] == '1':
        col.fillna(col.value_counts().index[1], inplace=True) # ensure we don't fill heterozygous
    else:
        col.fillna(col.value_counts().index[0], inplace=True)
    return col

def imputeData(df):
    df = df.replace('*', np.NaN)
    #df = df.astype('int64')
    #imputer = SimpleImputer(missing_values=np.nan, strategy= 'most_frequent')
    #return pd.DataFrame(imputer.fit_transform(df))
    return df.apply(lambda col:fillna(col))

def splitDF(df, numPieces): 
    return np.array_split(df, numPieces)
    
def imputeData2(df, chunkSize):
    df = df.replace('*', np.NaN)
    splits = splitDF(df, len(df) // chunkSize + 1) # splits data frame into pieces of length 100
    for dfi in splits:
        dfi = dfi.T
        dfi.apply(lambda col:fillna(col)) # fill with most common SNP value for that chunk
    return splits #returns list of split up data frames

def deleteDups(g):
    # goal is to eliminate duplicate genotypes/duplicate haplotypes
    g.drop_duplicates(inplace=True)
    return g

In [3]:
# Compatibility checker function
def checkPhase(g, h1, h2):
    # want to see if element wise sum of h1 and h2 is g
    # takes list g of SNPs
    # takes lists h1, h2 of SNPs
    import numpy as np
    g = np.array(g)
    h1 = np.array(h1)
    h2 = np.array(h2)
    comparison =  (h1 + h2 == g)
    return comparison.all()

def findDifference(g, h1): 
    # so that we can fill in a new haplo if we don't find a compatibile pair in Clark's
    g = np.array(g)
    h1 = np.array(h1)
    return g - h1


In [4]:
def clarks(genotypes):
    #input: genotype dataframe
    
    # need to give a starting pool
    # we can do this by phasing all of the deterministic genotypes
    genotypes = genotypes.astype('int64')
    haplotypes = []

    toDrop = [] # deterministic
    for ind in range(len(genotypes)):
        h = []
        g = genotypes.iloc[ind]
        for i in range(len(g)):
            if g[i] == 1: #non deterministic
                break
            if g[i] == 0:
                h.append(0)
            if g[i] == 2:
                h.append(1)
        if len(h) == len(g): # did you make it to the end of the string
            haplotypes.append(h)
            toDrop.append(ind) #thins out our new genotype list
    genotypes = genotypes.drop(genotypes.index[toDrop])

    for i,g in genotypes.iterrows():
        phased = False #flag variable
        for h1 in range(len(haplotypes)):
            for h2 in range(h1, len(haplotypes)):
                if checkPhase(g,haplotypes[h1],haplotypes[h2]): # we already have the phase accounted for
                    phased = True
        if phased == False: # now we need to add a haplo that works
            for h in haplotypes:
                diff = findDifference(g, h)
                # now just need to make sure this difference has no weird values -- i.e. is a valid addition
                if sum(0 <= x <= 1 for x in diff) == len(g):
                    haplotypes.append(h)
                    break
    return haplotypes

In [12]:
def clarksSplit(genotypes):
    #input: list of split up dataframes
    all_haps = []
    # need to give a starting pool
    # we can do this by phasing all of the deterministic genotypes
    last_ind = 0
    
    for dfindex in range(len(genotypes)):
        #print("index of subdataframe: ", dfindex)
        dfi = genotypes[dfindex]
        dfi = dfi.T
        dfi = dfi.astype('int64')
        index_update = 0
        #print("shape of subdata frame: ", dfi.shape)
        haplotypes = []
        toDrop = [] # deterministic
        for ind in range(len(dfi)):
            #print("current individual: ", ind)
            h = []
            g = dfi.iloc[ind]
            index_update = len(g)
            for i in range(len(g)):
                print("length of partial genotype ", len(g))
                #adj_i = (chunkSize*dfindex + i)# need to adjust indexing
                adj_i = last_ind + i 
                print("adjusted index", adj_i)
                if g[adj_i] == 1: #non deterministic
                    break
                if g[adj_i] == 0:
                    h.append(0)
                if g[adj_i] == 2:
                    h.append(1)
            if len(h) == len(g): # did you make it to the end of the string
                haplotypes.append(h)
                toDrop.append(ind) #thins out our new genotype list
        last_ind += index_update
        #print("last index", last_ind)
        dfi = dfi.drop(dfi.index[toDrop])

        for i,g in dfi.iterrows():
            phased = False #flag variable
            for h1 in range(len(haplotypes)):
                for h2 in range(h1, len(haplotypes)):
                    if checkPhase(g,haplotypes[h1],haplotypes[h2]): # we already have the phase accounted for
                        phased = True
            if phased == False: # now we need to add a haplo that works
                for h in haplotypes:
                    diff = findDifference(g, h)
                    # now just need to make sure this difference has no weird values -- i.e. is a valid addition
                    if sum(0 <= x <= 1 for x in diff) == len(g):
                        haplotypes.append(h)
                        break
        all_haps.append(haplotypes)
    return all_haps

In [6]:
def wrapper(genotypes):
    # do it all
    #genotypes = genotypes.T
    genotypes = imputeData(genotypes) # impute
    genotypes = genotypes.astype('int64')
    deleteDups(genotypes) # get rid of duplicates
    return clarks(genotypes)

In [7]:
ex1 = pd.read_csv("assignment/example_data_1_masked.txt", sep = " ", header=None)
ex1.shape

(39496, 50)

In [8]:
imputed = imputeData2(ex1, 5)

In [10]:
#check accuracy
ex1.un = pd.read_csv("assignment/example_data_1.txt", sep = " ", header = None)
imputedFull = pd.concat(imputed)
imputedFull = imputedFull.astype('int64')
booleanMaskDisc = pd.DataFrame(imputedFull == ex1.un)
maskedCorrectlyCounts = booleanMaskDisc.apply(pd.Series.value_counts, axis = 0)
np.array(maskedCorrectlyCounts.iloc[0])/(np.array(maskedCorrectlyCounts.iloc[1]) + np.array(maskedCorrectlyCounts.iloc[0]))

array([0.99407535, 0.98817602, 0.98987239, 0.99326514, 0.99415131,
       0.99415131, 0.99450577, 0.99399939, 0.99475896, 0.99468301,
       0.99432854, 0.9948096 , 0.99450577, 0.99402471, 0.9944045 ,
       0.99382216, 0.99508811, 0.99377152, 0.99478428, 0.99412599,
       0.99448045, 0.99501215, 0.99486024, 0.99417663, 0.99405003,
       0.99432854, 0.99448045, 0.99415131, 0.99501215, 0.99437918,
       0.9953413 , 0.99405003, 0.99445513, 0.99410067, 0.99554385,
       0.99415131, 0.99326514, 0.99402471, 0.99377152, 0.9937462 ,
       0.99356897, 0.99311323, 0.99455641, 0.99473364, 0.99463237,
       0.99470832, 0.99455641, 0.99402471, 0.99554385, 0.99465769])

In [187]:
imputedFull.shape

(39496, 50)

In [188]:
imputedFull = imputedFull.T
imputedFull.shape

(50, 39496)

In [189]:
deleteDups(imputedFull)
imputedFull.shape

(50, 39496)

In [202]:
#these stats only for 70
len(imputed) # 565 sub data frames for 70
d = imputed[1]
d = d.T
d.shape #50 individuals on the rows, 70 SNPs on the columms
g = d.iloc[1]
g = g.astype('int64')
len(g) #70 SNPs per person, switches to 69 at some point
g

70     0
71     0
72     0
73     2
74     0
75     0
76     2
77     0
78     2
79     2
80     2
81     0
82     0
83     2
84     2
85     1
86     0
87     2
88     0
89     1
90     1
91     2
92     1
93     2
94     1
95     0
96     1
97     1
98     1
99     1
      ..
110    2
111    1
112    2
113    1
114    0
115    1
116    0
117    1
118    2
119    2
120    2
121    1
122    2
123    2
124    0
125    0
126    0
127    2
128    0
129    2
130    0
131    1
132    2
133    0
134    0
135    0
136    2
137    0
138    1
139    1
Name: 1, Length: 70, dtype: int64

In [ ]:
trialrun_1 = clarksSplit(imputed)

In [ ]:
len(trialrun_1)

In [30]:
n = 0
for i in trialrun_1:
    if not len(i)  == 0:
        n+=1
        
n
    # chunk size 100 g_ives 200 subarrays' phases 
    # chunk size 70 gives 366 subarrays' phases
    # chunk size 40 gives 820/988 subarrays' phases
    # chunk size 15 gives 2593/2634 subarrays' phases
    # chunk size 10 gives 3935/3950 subarrays' phases
    # chunk size 7 gives 5642/5643 phasings!
    # chunk size 5 gives 7900/7900 phasings!
    
    
### KEEP TESTING!
## Smaller chunk sizes
    ## we do this simply by passing the right chunk size into the imputer
    ## after that its just a matter of running clarksSplit

7900

In [26]:
# find a way to combine these phasingsn
# psuedocode:
# for i from 0 to number of rows in largest subarray
    # tack on each subarray's [i%subarray length] row

# number of phasings we should get in total  == the number of rows in the largest subarray

maxlen = 0
minlen = 0
for i in trialrun_1:
    if i == 0: 
        minlen = len(i)
    length = len(i)
    if length > maxlen:
        maxlen = length
minlen #50

    

0

In [23]:
x = [[] for i in range(3)]
x[1].append(0)
x

[[], [0], []]

In [46]:

haplotypes = [[] for i in range(maxlen)]

for i in range(maxlen):
    for j in trialrun_1:
        haplotypes[i].extend(j[i%len(j)])
    

[[0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
